ใน ep ที่แล้ว 

# 0. Install

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Install Library ที่จำเป็น

In [69]:
## Colab
! curl -s https://course.fast.ai/setup/colab | bash

Updating fastai...
Done.


# 1. Import

Import Library ที่จะใช้ ในที่นี้คือ fastai และ fastai.text 

In [0]:
from fastai import *
from fastai.text import *

In [0]:
# import fastai.utils.collect_env
# fastai.utils.collect_env.show_install()

# 2. Dataset

ในเคสนี้เราจะใช้ IMDB Movie Review เป็นรีวิวหนังจากเว็บ IMDB ที่มีข้อความ และ คะแนนว่าเป็นแง่บวก หรือแง่ลบ เหมือนใน ep ก่อน ๆ 

ในการพัฒนา เราจะใช้ Dataset ชุดเล็กก่อน จะได้เร็ว เมื่อเทสทุกอย่างเรียบร้อย แล้วจึงขยับไปใช้ Dataset ชุดเต็ม 

In [72]:
path = untar_data(URLs.IMDB_SAMPLE)
path.ls()

[PosixPath('/content/data/imdb_sample/texts.csv')]

# 3. Preprocessing

## 3.1 Data Block API

เราจะใช้ [Data Block API](https://www.bualabs.com/archives/2693/data-block-api-data-pipeline-machine-learning-supervised-learning-preprocessing-ep-5/) กำหนด Data Pipeline ทีละขั้น Tokenization, Numericalization, Split, Labeling ก็จะทำให้ยืดหยุ่นมากขึ้น

In [0]:
data_lm = (TextList.from_csv(path, 'texts.csv', cols='text')
                .split_from_df(col=2)
                .label_from_df(cols=0)
                .databunch())

ดูตัวอย่างคำศัพท์ใน vocab Dictionary

In [78]:
data_lm.vocab.itos[:20]

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 '.',
 ',',
 'and',
 'a',
 'of',
 'to',
 'is',
 'it',
 'in',
 'i']

ดูตัวอย่างข้อมูล text

In [79]:
data_lm.train_ds[0][0]

Text xxbos xxmaj un - xxunk - believable ! xxmaj meg xxmaj ryan does n't even look her usual xxunk lovable self in this , which normally makes me forgive her shallow xxunk acting xxunk . xxmaj hard to believe she was the producer on this dog . xxmaj plus xxmaj kevin xxmaj kline : what kind of suicide trip has his career been on ? xxmaj xxunk ... xxmaj xxunk ! ! ! xxmaj finally this was directed by the guy who did xxmaj big xxmaj xxunk ? xxmaj must be a replay of xxmaj jonestown - hollywood style . xxmaj xxunk !

ดูข้อมูลภายใน ในรูปแบบตัวเลข

In [80]:
data_lm.train_ds[0][0].data[:10]

array([   2,    5, 4619,   25,    0,   25,  867,   52,    5, 3776])

# 2/2. Full Dataset

Download ชุดข้อมูลตัวเต็ม

In [81]:
path = untar_data(URLs.IMDB)
path.ls()

[PosixPath('/content/data/imdb/lm_databunch'),
 PosixPath('/content/data/imdb/train'),
 PosixPath('/content/data/imdb/test'),
 PosixPath('/content/data/imdb/unsup'),
 PosixPath('/content/data/imdb/README'),
 PosixPath('/content/data/imdb/tmp_clas'),
 PosixPath('/content/data/imdb/tmp_lm'),
 PosixPath('/content/data/imdb/imdb.vocab')]

In [82]:
(path/'train').ls()

[PosixPath('/content/data/imdb/train/pos'),
 PosixPath('/content/data/imdb/train/labeledBow.feat'),
 PosixPath('/content/data/imdb/train/neg'),
 PosixPath('/content/data/imdb/train/unsupBow.feat')]

# 3/2. Preprocessing

Data Pipeline ด้วย Data Block API

In [0]:
# bs=48
# bs=24
bs=192

In [0]:
data_lm = (TextList.from_folder(path)
                    .filter_by_folder(include=['train', 'test', 'unsup'])
                    .split_by_rand_pct(0.1)
                    .label_for_lm()
                    .databunch(bs=bs, num_workers=1))

ในข้อมูล 90,000 Record มีคำศัพท์ 60,000 คำ (Default)

In [85]:
len(data_lm.vocab.itos), len(data_lm.train_ds)

(60000, 90000)

In [86]:
data_lm.show_batch()

idx,text
0,"xxmaj jack xxmaj bauer is a federal agent who is assigned the protection of the senator xxmaj david xxmaj palmer . xxmaj he ca n't trust in anybody because people of the xxup ctu may be involved . xxmaj and , when this events occurred his daughter : xxmaj kimberly escapes from house to a party . xxmaj but ... \n \n xxmaj at the end of the episode"
1,"would sell , so why do n't they make a movie out of my bride seeking adventures in xxmaj russia ? xxmaj however , there is one factual impossibility in this film , and that is the way which the guy orders his bride from a catalog and having her arrive at an airport . xxmaj it does n't work that way at all , so i do n't understand"
2,"be xxmaj bridget xxmaj hennessy , played by xxmaj kaley xxmaj cuoco . xxmaj she is the ultimate blonde : gorgeous , slow , dim - witted , yet she is a smart person . xxmaj she is off the wall hilarious with her innocent ' blonde ' humor and how conceited she is . \n \n xxmaj amy xxmaj davidson can get a tad annoying as xxmaj kerry"
3,"with the police with the fascination of a foreigner , but i think that adds something rather than detracting from the film . xxmaj it 's not true that hippies did not exist in xxmaj italy at the time ( think of the ' 68 protests , like in xxmaj france ) , although they were obviously different from the xxmaj american counterpart and in some ways even more militant"
4,"remake of it . xxmaj even though it did n't follow the original storyline ( that 's why i gave it 8 out of 10 ) , it was still pretty good and this is probably a better storyline . \n \n xxmaj as for the car , well the xxup dj 's comment at the end about the xxmaj challenger going 185 mph into the bulldozers is pretty"


เซฟไว้ก่อน 

In [0]:
data_lm.save('lm_databunch')

โหลด Language Model DataBunch ที่เซฟไว้ แล้วเช็คข้อมูล

In [88]:
data_lm = load_data(path, 'lm_databunch', bs=bs)
len(data_lm.vocab.itos), len(data_lm.train_ds)

(60000, 90000)

# 4. Language Model จาก WikiText-103

สร้าง Language Model Learner ขึ้นมา โดยใช้ data_lm จาก IMDB ด้านบน และ สถาปัตยกรรม AWD_LSTM [Model Architecture](https://www.bualabs.com/archives/2703/how-to-read-model-convolutional-neural-network-shape-activation-map-model-architecture-convnet-ep-7/) โดยที่ยังไม่ได้เริ่มต้นเทรน

AWD_LSTM ได้ถูก Pre-Train กับ WikiText-103 Dataset เรียบร้อยแล้ว

In [0]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)
# learn_lm

In [91]:
learn_lm.model[0]

AWD_LSTM(
  (encoder): Embedding(60000, 400, padding_idx=1)
  (encoder_dp): EmbeddingDropout(
    (emb): Embedding(60000, 400, padding_idx=1)
  )
  (rnns): ModuleList(
    (0): WeightDropout(
      (module): LSTM(400, 1152, batch_first=True)
    )
    (1): WeightDropout(
      (module): LSTM(1152, 1152, batch_first=True)
    )
    (2): WeightDropout(
      (module): LSTM(1152, 400, batch_first=True)
    )
  )
  (input_dp): RNNDropout()
  (hidden_dps): ModuleList(
    (0): RNNDropout()
    (1): RNNDropout()
    (2): RNNDropout()
  )
)

In [92]:
learn_lm.model[0].encoder

Embedding(60000, 400, padding_idx=1)

In [93]:
Config().model_path().ls()

[PosixPath('/root/.fastai/models/wt103-fwd.tgz'),
 PosixPath('/root/.fastai/models/wt103-fwd')]

In [94]:
(Config().model_path()/'wt103-fwd').ls()

[PosixPath('/root/.fastai/models/wt103-fwd/lstm_fwd.pth'),
 PosixPath('/root/.fastai/models/wt103-fwd/itos_wt103.pkl')]

## 4.1 vocab Dictionary ของ WikiText-103 

โหลด Dictionary ของ WikiText-103 ขึ้นมา 

Language Model ต้องมาพร้อมกับ vocab Dictionary เสมอ ไม่เช่นนั้นเราจะไม่รู้ว่า คำศัพท์ไหนเป็นคำไหน เพราะในโมเดลจะเป็นตัวเลข Index / Embedding

In [0]:
wiki_itos = pickle.load(open(Config().model_path()/'wt103-fwd/itos_wt103.pkl', 'rb'))

ดูจำนวนคำศัพท์ ตัวอย่างข้อมูล หัว / ท้าย

In [22]:
len(wiki_itos), wiki_itos[:20], wiki_itos[-20:]

(60000,
 ['xxunk',
  'xxpad',
  'xxbos',
  'xxeos',
  'xxfld',
  'xxmaj',
  'xxup',
  'xxrep',
  'xxwrep',
  'the',
  ',',
  '.',
  'of',
  'and',
  'in',
  'to',
  'a',
  '=',
  '"',
  'was'],
 ['repaying',
  'ak-47',
  'cultic',
  'jaume',
  'leonis',
  'subtracting',
  'sexed',
  'bursa',
  '-style',
  'lithographs',
  'wgbh',
  'subsystem',
  '831',
  'lifelines',
  'bergeron',
  'adaptions',
  'slurs',
  'acasta',
  'stuccoed',
  'spellbound'])

## 4.2 vocab Dictionary ของ IMDB

เปรียบเทียบ 2 vocab Dictionary

In [0]:
vocab = data_lm.vocab

In [0]:
imdb_itos = data_lm.vocab.itos

In [25]:
vocab.stoi['love']

134

In [26]:
imdb_itos[vocab.stoi['love']]

'love'

In [27]:
imdb_itos[vocab.stoi['Sukhumvit']]

'xxunk'

In [0]:
awd = learn_lm.model[0]

In [0]:
from scipy.spatial.distance import cosine as dist

In [0]:
enc = learn_lm.model[0].encoder

In [31]:
enc.weight.size()

torch.Size([60000, 400])

## 4.3 Difference in vocab between IMDB and Wikipedia

ขนาด vocab เท่ากัน

In [32]:
len(wiki_itos), len(imdb_itos)

(60000, 60000)

แต่ข้างใน ไม่เหมือนกัน เปรียบเทียบ 10 คำศัพท์สุดท้าย

In [33]:
wiki_itos[-10:], imdb_itos[-10:]

(['wgbh',
  'subsystem',
  '831',
  'lifelines',
  'bergeron',
  'adaptions',
  'slurs',
  'acasta',
  'stuccoed',
  'spellbound'],
 ["o'daniel",
  'blueish',
  'mc5',
  'fennie',
  'canby',
  'coherant',
  'lohde',
  'backwoodsman',
  'maguires',
  '24-minute'])

In [0]:
wiki_words = set(wiki_itos)
imdb_words = set(imdb_itos)

In [35]:
wiki_not_imdb = wiki_words.difference(imdb_words)
len(wiki_not_imdb), [val for i, val in enumerate(itertools.islice(wiki_not_imdb, 20))]

(24225,
 ['éditions',
  'jex',
  'sabor',
  'ferugliotherium',
  'hermansen',
  'afa',
  'keswick',
  'phonemes',
  'republike',
  '232',
  'tapers',
  'transjordan',
  'coley',
  'wetherall',
  'kawada',
  'culverhouse',
  'sorrel',
  'conspecific',
  'synapomorphies',
  'm31'])

In [36]:
imdb_not_wiki = imdb_words.difference(wiki_words)
len(imdb_not_wiki), [val for i, val in enumerate(itertools.islice(imdb_not_wiki, 20))]

(24225,
 ['compensations',
  'higurashi',
  'premonitions',
  'seamed',
  'hichcock',
  'syrupy',
  'tallahatchie',
  'intergenerational',
  'masina',
  'baffles',
  'zantara',
  'boytown',
  'solipsistic',
  'spanner',
  'misdirects',
  'arrondissements',
  'crapped',
  'akhurst',
  'alvaro',
  'cathleen'])

## 4.4 Out of vocab Initialization

คำว่า cpu อยู่ใน vocab ของ WikiText-103 แต่ไม่อยู่ใน IMDB

In [37]:
w = "cpu"
vocab.stoi[w], w in wiki_words, w in imdb_words

(0, True, False)

คำเหล่านี้ อยู่ใน vocab ของ IMDB แต่ไม่อยู่ใน WikiText-103 เราจะ Initialize Embedding ของคำเหล่านี้ แบบ Random

In [38]:
w = "senselessness"
vocab.stoi[w], w in wiki_words, w in imdb_words

(38883, False, True)

In [39]:
w = "unthinking"
vocab.stoi[w], w in wiki_words, w in imdb_words

(26965, False, True)

In [40]:
w = "forklift"
vocab.stoi[w], w in wiki_words, w in imdb_words

(43635, False, True)

Initialize ด้วยค่า Random เล็ก ๆ ใกล้เคียงกัน

In [41]:
np.allclose(enc.weight[vocab.stoi['senselessness'], :], 
            enc.weight[vocab.stoi['unthinking'], :])

True

แต่ถ้าเทียบกับ Embedding ที่เทรนเรียบร้อยแล้ว ก็จะต่างกันมาก

In [42]:
np.allclose(enc.weight[vocab.stoi['senselessness'], :], 
            enc.weight[vocab.stoi['capital'], :])

False

ดู Embedding

In [43]:
vocab.stoi['senselessness'], vocab.stoi['unthinking'], vocab.stoi['capital'], vocab.stoi['cpu']

(38883, 26965, 5777, 0)

In [44]:
len(enc.weight[vocab.stoi['senselessness'], :]), enc.weight[vocab.stoi['senselessness'], :][:50]

(400,
 tensor([ 0.0253,  0.0026,  0.0423, -0.0274, -0.0110, -0.0100, -0.0628,  0.1794,
         -0.1275, -0.2096, -0.0124, -0.0056, -0.0505, -0.0501, -0.0428,  0.1726,
         -0.0650, -0.0571, -0.1103,  0.0186,  0.0053, -0.0059,  0.0299,  0.0192,
         -0.0240,  0.1432, -0.1064, -0.0097,  0.0100, -0.0399,  0.0282,  0.0319,
          0.1297,  0.0687, -0.0047,  0.0168,  0.0201,  0.1346, -0.0336, -0.0449,
          0.0265, -0.0756, -0.0225,  0.0196,  0.0221,  0.0214,  0.0077, -0.1311,
         -0.1357, -0.0128], device='cuda:0', grad_fn=<SliceBackward>))

In [45]:
len(enc.weight[vocab.stoi['unthinking'], :]), enc.weight[vocab.stoi['unthinking'], :][:50]

(400,
 tensor([ 0.0253,  0.0026,  0.0423, -0.0274, -0.0110, -0.0100, -0.0628,  0.1794,
         -0.1275, -0.2096, -0.0124, -0.0056, -0.0505, -0.0501, -0.0428,  0.1726,
         -0.0650, -0.0571, -0.1103,  0.0186,  0.0053, -0.0059,  0.0299,  0.0192,
         -0.0240,  0.1432, -0.1064, -0.0097,  0.0100, -0.0399,  0.0282,  0.0319,
          0.1297,  0.0687, -0.0047,  0.0168,  0.0201,  0.1346, -0.0336, -0.0449,
          0.0265, -0.0756, -0.0225,  0.0196,  0.0221,  0.0214,  0.0077, -0.1311,
         -0.1357, -0.0128], device='cuda:0', grad_fn=<SliceBackward>))

In [46]:
len(enc.weight[vocab.stoi['capital'], :]), enc.weight[vocab.stoi['capital'], :][:50]

(400,
 tensor([-0.2566,  0.1062,  0.3081,  0.2742,  0.1667, -0.1626, -0.5122, -0.1261,
          0.4548, -0.4177, -0.1565,  0.0752,  0.2253,  0.2031, -0.3125,  0.0310,
          0.5112, -0.3691, -0.2947, -0.0168, -0.1147,  0.3074, -0.1654, -0.2202,
          0.0538, -0.4041,  0.1666, -0.0109, -0.2803,  0.3044, -0.2279,  0.1786,
         -0.2502, -0.2334,  0.4438,  0.2404,  0.1746,  0.1509,  0.0351,  0.2844,
         -0.0244, -0.2031,  0.1228, -0.2357, -0.2065,  0.5811,  0.1338,  0.4087,
         -0.5034, -0.0150], device='cuda:0', grad_fn=<SliceBackward>))

# สรุป

* เราได้เรียนรู้ว่า 

# Credit

* https://www.youtube.com/watch?v=dt7sArnLo1g&list=PLtmWHNX-gukKocXQOkQjuVxglSDYWsSh9&index=6&t=0s
* https://www.bualabs.com/archives/2693/data-block-api-data-pipeline-machine-learning-supervised-learning-preprocessing-ep-5/
* https://www.bualabs.com/archives/926/sentiment-analysis-imdb-movie-review-ulmfit-sentiment-analysis-ep-1/
* https://www.bualabs.com/archives/3000/sentiment-classification-imdb-movie-reviews-with-naive-bayes-logistic-regression-nlp-ep-5/
* https://docs.fast.ai/data_block.html
* https://docs.fast.ai/text.data.html
* https://forums.fast.ai/t/language-model-zoo-gorilla/14623
* https://www.imdb.com/interfaces/
* https://blog.einstein.ai/the-wikitext-long-term-dependency-language-modeling-dataset/
* https://arxiv.org/abs/1801.06146
* 